In [1]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [1]:
!wget https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q3.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q4.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2020Q1.csv.zip

--2021-10-26 20:39:53--  https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 54.68.178.152, 52.41.216.117, 54.68.47.150
Connecting to resources.lendingclub.com (resources.lendingclub.com)|54.68.178.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2019Q1.csv.zip.1’

LoanStats_2019Q1.cs     [             <=>    ]  20.27M  5.38MB/s    in 4.3s    

2021-10-26 20:39:58 (4.76 MB/s) - ‘LoanStats_2019Q1.csv.zip.1’ saved [21257309]

--2021-10-26 20:39:58--  https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 54.68.47.150, 52.41.216.117, 54.68.178.152
Connecting to resources.lendingclub.com (resources.lendingclub.com)|54.68.47.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2019Q2.csv.zip.1

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", 
    "verification_status", "pymnt_plan", "dti", "delinq_2yrs", 
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", 
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt", 
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", 
    "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
    "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", 
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", 
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", 
    "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", 
    "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", 
    "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", 
    "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", 
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", 
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", 
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", 
    "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", 
    "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit", 
    "hardship_flag", "debt_settlement_flag",
    "loan_status"
]

target = "loan_status"

In [7]:
# Load the data
df1 = pd.read_csv(Path('LoanStats_2019Q1a.csv.zip'), skiprows=1)[:-2]
df2 = pd.read_csv(Path('LoanStats_2019Q2a.csv.zip'), skiprows=1)[:-2]
df3 = pd.read_csv(Path('LoanStats_2019Q3a.csv.zip'), skiprows=1)[:-2]
df4 = pd.read_csv(Path('LoanStats_2019Q4a.csv.zip'), skiprows=1)[:-2]

df = pd.concat([df1, df2, df3, df4]).loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)


low_risk_rows = df[df[target] == 'low_risk']
high_risk_rows = df[df[target] == 'high_risk']

#df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
df = df.rename({target:'target'}, axis="columns")
df

/Users/kshimberg/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/kshimberg/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,36000.0,0.0756,1120.82,MORTGAGE,184000.0,Not Verified,n,22.55,0.0,2.0,...,21.7,0.0,0.0,390746.0,119845.0,146700.0,50735.0,N,N,low_risk
1,38275.0,0.1774,966.53,MORTGAGE,47000.0,Verified,n,54.29,0.0,1.0,...,71.4,0.0,0.0,195807.0,63732.0,50200.0,15657.0,N,N,low_risk
2,12000.0,0.0881,380.54,RENT,55000.0,Source Verified,n,7.59,0.0,0.0,...,0.0,0.0,0.0,18775.0,12084.0,8000.0,10775.0,N,N,low_risk
3,10000.0,0.0819,314.25,RENT,58000.0,Not Verified,n,10.37,0.0,0.0,...,16.7,0.0,0.0,60318.0,24309.0,35600.0,17418.0,N,N,low_risk
4,30000.0,0.1171,662.95,MORTGAGE,78000.0,Not Verified,n,16.22,0.0,0.0,...,25.0,0.0,0.0,274438.0,33876.0,19100.0,24807.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13309,16000.0,0.1033,342.56,RENT,54000.0,Not Verified,n,26.93,0.0,1.0,...,66.7,0.0,0.0,59690.0,34904.0,14400.0,27690.0,N,N,high_risk
13310,25000.0,0.1171,552.46,RENT,120000.0,Not Verified,n,8.94,0.0,1.0,...,0.0,0.0,0.0,77424.0,32017.0,51500.0,10000.0,N,N,high_risk
13311,14400.0,0.1430,494.26,MORTGAGE,75000.0,Not Verified,n,22.50,0.0,0.0,...,50.0,0.0,0.0,305090.0,36244.0,15100.0,53974.0,N,N,high_risk
13312,40000.0,0.0646,1225.24,MORTGAGE,92108.0,Verified,n,17.29,0.0,0.0,...,16.7,0.0,0.0,405077.0,75492.0,116300.0,12261.0,N,N,high_risk


In [8]:
df.to_csv('2019loansa.csv', index=False)

In [9]:
# Load the data
validate_df = pd.read_csv(Path('LoanStats_2020Q1a.csv.zip'), skiprows=1)[:-2]
validate_df = validate_df.loc[:, columns].copy()

# Drop the null columns where all values are null
validate_df = validate_df.dropna(axis='columns', how='all')

# Drop the null rows
validate_df = validate_df.dropna()

# Remove the `Issued` loan status
issued_mask = validate_df[target] != 'Issued'
validate_df = validate_df.loc[issued_mask]

# convert interest rate to numerical
validate_df['int_rate'] = validate_df['int_rate'].str.replace('%', '')
validate_df['int_rate'] = validate_df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = dict.fromkeys(['Current', 'Fully Paid'], 'low_risk') 
validate_df = validate_df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period', 'Charged Off'], 'high_risk')    
validate_df = validate_df.replace(x)

low_risk_rows = validate_df[validate_df[target] == 'low_risk']
high_risk_rows = validate_df[validate_df[target] == 'high_risk']

validate_df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=37), high_risk_rows])
validate_df = validate_df.reset_index(drop=True)
validate_df = validate_df.rename({target:'target'}, axis="columns")
validate_df

/Users/kshimberg/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,26400.0,0.1033,565.22,MORTGAGE,70000.0,Not Verified,n,24.48,0.0,0.0,...,0.0,0.0,0.0,258000.0,38985.0,48100.0,30000.0,N,N,low_risk
1,25000.0,0.0819,785.61,MORTGAGE,100000.0,Verified,n,32.76,0.0,2.0,...,0.0,0.0,0.0,427746.0,97284.0,51200.0,116246.0,N,N,low_risk
2,10200.0,0.1430,350.10,MORTGAGE,40000.0,Not Verified,n,20.67,0.0,0.0,...,100.0,0.0,0.0,28204.0,9242.0,2500.0,24204.0,N,N,low_risk
3,15000.0,0.2305,423.29,MORTGAGE,58500.0,Not Verified,n,18.99,0.0,2.0,...,25.0,0.0,0.0,44700.0,30868.0,20000.0,23000.0,N,N,low_risk
4,40000.0,0.1102,870.10,MORTGAGE,90000.0,Not Verified,n,38.87,0.0,2.0,...,0.0,0.0,0.0,322216.0,179158.0,63400.0,174616.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8141,10000.0,0.2305,387.36,RENT,33000.0,Verified,n,38.51,0.0,2.0,...,0.0,0.0,0.0,43250.0,33022.0,8500.0,29550.0,N,N,high_risk
8142,8000.0,0.1862,205.86,RENT,38000.0,Source Verified,n,16.36,0.0,1.0,...,0.0,1.0,0.0,31357.0,19595.0,1500.0,9657.0,N,N,high_risk
8143,30000.0,0.2055,1123.34,RENT,180000.0,Source Verified,n,12.06,0.0,0.0,...,33.3,0.0,0.0,218686.0,209389.0,5600.0,209986.0,N,N,high_risk
8144,25000.0,0.2565,1002.62,RENT,60000.0,Source Verified,n,22.44,0.0,0.0,...,62.5,0.0,0.0,57163.0,42830.0,16000.0,41163.0,N,N,high_risk


In [10]:
validate_df.to_csv('2020Q1loansa.csv', index=False)